In [83]:
import torch.nn.functional as F
import pytorch_lightning as pl
import pickle
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader, random_split
from CustomDataModule import CustomDataModule
from LSTMModel import LSTMModel
from lightning.pytorch.loggers import WandbLogger
from pytorch_lightning.utilities.model_summary import summarize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [84]:
def process_radar_return(radar_return):
    # Ensure the radar return has shape (num_rows, num_cols)
    assert len(radar_return.shape) == 2

    # Define the target length of 512
    target_length = 512

    # Randomly select the starting index for the sequence
    start_index = np.random.randint(0, radar_return.shape[1] - 5)

    # Select the subsequent 4 indices to form a sequence of 5 adjacent pulses
    selected_pulses = np.arange(start_index, start_index + 6)

    # Concatenate selected pulses along rows
    concatenated_pulses = np.concatenate([radar_return[:, pulse] for pulse in selected_pulses], axis=0)

    # Take np.abs to convert complex numbers to real numbers
    epsilon = 1e-10
    processed_radar_return = concatenated_pulses # 10 * np.log10(((np.abs(concatenated_pulses))**2)+epsilon)

    # Ensure the processed radar return has length 512
    if processed_radar_return.shape[0] > target_length:
        # If length is greater than 512, truncate the vector
        processed_radar_return = processed_radar_return[:target_length]
    elif processed_radar_return.shape[0] < target_length:
        # If length is less than 512, pad with zeros
        print("error: default length is shorter than min length")
        # min_value = np.min(processed_radar_return)
        # pad_value = -200
        # processed_radar_return = np.pad(processed_radar_return,
        #                                 (0, target_length - processed_radar_return.shape[0]),
        #                                 mode='constant', constant_values=pad_value)

    return processed_radar_return

In [85]:
def normalize_radar_return_column(combined_df):
    radar_data = combined_df['radar_return']

    # Flatten the radar data for scaler fitting
    flattened_data = [np.array(row).flatten() for row in radar_data]
    flattened_data = np.concatenate(flattened_data).reshape(-1, 1)

    # Fit the scaler on the flattened data
    scaler = MinMaxScaler()
    scaler.fit(flattened_data)

    # Transform each row separately
    normalized_data = [scaler.transform(np.array(row).reshape(-1, 1)).flatten().tolist() for row in radar_data]

    # Reshape the normalized data back to its original shape
    combined_df['radar_return'] = normalized_data

    return combined_df

In [86]:
data_dir = 'First_data'  # Change this to your data folder path
dataframes = []

# set device!
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

True


In [87]:
print("creating the dataframe...")
for filename in os.listdir(data_dir):
    if filename.endswith('.pickle'):
        file_path = os.path.join(data_dir, filename)
        with open(file_path, 'rb') as f:
            # Load data from pickle file
            data = pickle.load(f)
            # Extract radar_return and object_id
            radar_return = data['radar_return']
            object_id = data['object_id']
            # Concatenate radar_return along the columns
            concatenated_radar = process_radar_return(radar_return).astype('float32')
            # Create a DataFrame with concatenated radar and object_id
            df = pd.DataFrame({'radar_return': [concatenated_radar], 'object_id': [object_id]})
            # Append the DataFrame to the list
            dataframes.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dataframes[::50], ignore_index=True)
#combined_df = normalize_radar_return_column(combined_df)
print(normalize_radar_return_column(combined_df))

creating the dataframe...
                                          radar_return  \
0    [0.002982319099828601, 0.008598754182457924, 0...   
1    [0.0030343777034431696, 0.013289732858538628, ...   
2    [0.0032143485732376575, 0.0031584021635353565,...   
3    [3.573842332116328e-05, 0.00010574382031336427...   
4    [0.00015461580187547952, 4.985860869055614e-05...   
..                                                 ...   
395  [8.021188114071265e-05, 0.00019797529967036098...   
396  [0.0008380835643038154, 0.0015876194229349494,...   
397  [8.306884410558268e-05, 7.893757720012218e-05,...   
398  [0.0006163278594613075, 0.0010234196670353413,...   
399  [0.00039249903056770563, 0.0001597660884726792...   

                            object_id  
0    1c47c37d6c785d4fe7fd9a18c19837e5  
1    1c47c37d6c785d4fe7fd9a18c19837e5  
2    1c47c37d6c785d4fe7fd9a18c19837e5  
3    1c47c37d6c785d4fe7fd9a18c19837e5  
4    1c47c37d6c785d4fe7fd9a18c19837e5  
..                                ...

In [88]:
class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        return self.model(x)

In [89]:
class RadarDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


In [94]:
radar_data = torch.tensor(combined_df['radar_return'].values.tolist(), dtype=torch.float32)
object_ids = combined_df['object_id'].values.tolist()
# Use LabelEncoder to convert object_ids to numerical labels
label_encoder = LabelEncoder()
object_ids_encoded = label_encoder.fit_transform(object_ids)
# print(radar_data[0])
# print(object_ids_encoded)

# Convert encoded labels to tensor
object_ids_tensor = torch.tensor(object_ids_encoded, dtype=torch.float32)
data_train, data_test, label_train, label_test = train_test_split(radar_data, object_ids_encoded, stratify=object_ids_encoded)

# Define hyperparameters
input_size = radar_data.shape[1]
hidden_size = 512
output_size = 10
learning_rate = 0.001
num_epochs = 500
batch_size = 32

# Create FNN instance
model = FNN(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create DataLoader for batch training
dataset = RadarDataset(data_train, label_train)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

val_dataset = RadarDataset(data_test, label_test)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [95]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to GPU if available
model.to(device)

FNN(
  (model): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=512, out_features=512, bias=True)
    (10): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Linear(in_features=512, out_features=512, bias=True)
    (13): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [96]:

# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_correct_predictions = 0
    train_total_samples = 0
    train_total_loss = 0.0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels.view(-1).long())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        train_correct_predictions += (predicted == labels).sum().item()
        train_total_samples += labels.size(0)

        train_total_loss += loss.item()

        avg_train_loss = train_total_loss / (i + 1)
        train_accuracy = train_correct_predictions / train_total_samples

        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')

    # Validation phase
    model.eval()
    val_correct_predictions = 0
    val_total_samples = 0
    val_total_loss = 0.0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels.view(-1).long())

            _, predicted = torch.max(outputs, 1)
            val_correct_predictions += (predicted == labels).sum().item()
            val_total_samples += labels.size(0)

            val_total_loss += loss.item()

    avg_val_loss = val_total_loss / len(val_loader)
    val_accuracy = val_correct_predictions / val_total_samples

    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

print('Training finished.')

Epoch [1/500], Step [1/10], Train Loss: 2.3344, Train Accuracy: 0.0938
Epoch [1/500], Step [2/10], Train Loss: 2.3717, Train Accuracy: 0.0625
Epoch [1/500], Step [3/10], Train Loss: 2.3929, Train Accuracy: 0.0729
Epoch [1/500], Step [4/10], Train Loss: 2.3987, Train Accuracy: 0.0703
Epoch [1/500], Step [5/10], Train Loss: 2.3950, Train Accuracy: 0.0750
Epoch [1/500], Step [6/10], Train Loss: 2.4050, Train Accuracy: 0.0781
Epoch [1/500], Step [7/10], Train Loss: 2.3840, Train Accuracy: 0.0938
Epoch [1/500], Step [8/10], Train Loss: 2.3774, Train Accuracy: 0.0938
Epoch [1/500], Step [9/10], Train Loss: 2.3799, Train Accuracy: 0.0972
Epoch [1/500], Step [10/10], Train Loss: 2.4203, Train Accuracy: 0.0967
Epoch [1/500], Validation Loss: 2.3195, Validation Accuracy: 0.1000
Epoch [2/500], Step [1/10], Train Loss: 2.2473, Train Accuracy: 0.2188
Epoch [2/500], Step [2/10], Train Loss: 2.3530, Train Accuracy: 0.1719
Epoch [2/500], Step [3/10], Train Loss: 2.3254, Train Accuracy: 0.1771
Epoch [2